In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#hyperparameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [ ]:
train = dsets.CIFAR10(root = '/content/drive/MyDrive/CSCE464/Datasets/',
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True)
test = dsets.CIFAR10(root = '/content/drive/MyDrive/CSCE464/Datasets/',
                    train=False,
                    transform=transforms.ToTensor(),
                    download=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Linear(8 * 8 * 64, 10)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out



In [ ]:
model = CNN().to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss() # Softmax is internall computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    pred = model(X).to(device)
    cost = loss(pred, Y).to(device)

    correct_prediction = torch.argmax(pred, 1) == Y
    accuracy = correct_prediction.float().mean()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("Epoch: {:4d} / {} Cost: {:.6f} Training Accuracy: {:.2f}%".format(epoch+1, training_epochs, cost.item(), accuracy*100))


print("Learning Finished!")

Epoch:    1 / 20 Cost: 1.166078 Training Accuracy: 57.00%
Epoch:    2 / 20 Cost: 1.283499 Training Accuracy: 52.00%
Epoch:    3 / 20 Cost: 1.177776 Training Accuracy: 53.00%
Epoch:    4 / 20 Cost: 0.891507 Training Accuracy: 76.00%
Epoch:    5 / 20 Cost: 0.914787 Training Accuracy: 68.00%
Epoch:    6 / 20 Cost: 0.840957 Training Accuracy: 69.00%
Epoch:    7 / 20 Cost: 0.687658 Training Accuracy: 75.00%
Epoch:    8 / 20 Cost: 0.913826 Training Accuracy: 71.00%
Epoch:    9 / 20 Cost: 0.645150 Training Accuracy: 80.00%
Epoch:   10 / 20 Cost: 0.697220 Training Accuracy: 80.00%
Epoch:   11 / 20 Cost: 0.633527 Training Accuracy: 74.00%
Epoch:   12 / 20 Cost: 0.617651 Training Accuracy: 79.00%
Epoch:   13 / 20 Cost: 0.730152 Training Accuracy: 77.00%
Epoch:   14 / 20 Cost: 0.735163 Training Accuracy: 74.00%
Epoch:   15 / 20 Cost: 0.601157 Training Accuracy: 76.00%
Epoch:   16 / 20 Cost: 0.689985 Training Accuracy: 77.00%
Epoch:   17 / 20 Cost: 0.627424 Training Accuracy: 77.00%
Epoch:   18 / 

In [ ]:
model.eval()
with torch.no_grad():
  for data in test_loader:
    x_test, y_test = data[0].to(device), data[1].to(device)
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
  print('Testing Accuracy: {:.2f}%'.format(accuracy.item()*100))

Testing Accuracy: 74.00%
